In [48]:
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torch.nn.init

from matplotlib.pyplot import imshow
%matplotlib inline

In [49]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.duca.manual_seed_all(777)

In [57]:
trans = transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root = 'data', transform = trans)

In [58]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers =2)

In [59]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1= nn.Sequential(
        nn.Conv2d(3,6,5),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(6,16,5),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        
        self.layer3= nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,3)
        )
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.layer3(out)
        return out

In [60]:
net= CNN().to(device)


In [63]:
loss_func = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)

In [64]:
#training
total_batch = len(data_loader)

epochs = 3
for epoch in range(epochs):
    avg_cost = 0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out,labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss /total_batch
    print('[Epoch : {}] cost = {}'.format(epoch+1, avg_cost))

[Epoch : 1] cost = 0.6105397343635559
[Epoch : 2] cost = 0.39476218819618225
[Epoch : 3] cost = 0.3120306432247162


In [72]:
test_data = torchvision.datasets.ImageFolder(root = 'data/test', transform = trans)

In [73]:
test_set = DataLoader(dataset= test_data, batch_size = len(test_data))

In [79]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, labels = data
        imgs = imgs.to(device)
        label = labels.to(device)
        
        prediction = net(imgs)
        correct_prediction = torch.argmax(prediction, 1) == label
        print('Accuracy : ', accuracy.item())

Accuracy :  0.96875
